In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AutoSxS：在 Vertex AI 模型注册中评估 LLM 与第三方模型相比

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/model_based_llm_evaluation/autosxs_llm_evaluation_for_summarization_task.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fmodel_evaluation%2Fmodel_based_llm_evaluation%2Fautosxs_llm_evaluation_for_summarization_task.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在企业 Colab 中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_evaluation/model_based_llm_evaluation/autosxs_llm_evaluation_for_summarization_task.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Workbench 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/model_based_llm_evaluation/autosxs_llm_evaluation_for_summarization_task.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上查看
    </a>
  </td>
</table>

## 概述

本笔记本演示了如何使用Vertex AI自动并行评估（AutoSxS）来评估Vertex AI模型注册中的生成式AI模型与第三方语言模型之间的性能。

AutoSxS是一个模型辅助评估工具，可以帮助您将两个大型语言模型（LLMs）进行并行比较。作为AutoSxS的预览版本，我们目前仅支持对摘要和问答任务进行模型比较。在未来，我们将支持更多任务和定制化。

了解更多关于[Vertex AI AutoSxS模型评估](https://cloud.google.com/vertex-ai/docs/generative-ai/models/side-by-side-eval#autosxs)。

### 目标

在本教程中，您将学习如何使用`Vertex AI Pipelines`和`google_cloud_pipeline_components`来评估两个LLM模型之间的性能：

本教程使用以下谷歌云机器学习服务和资源：

- 云存储
- Vertex AI PaLM API
- Vertex AI Pipelines
- Vertex AI Batch Prediction

执行的步骤包括：

- 从公共来源获取数据集。
- 在本地预处理数据并将测试数据保存在云存储中。
- 创建并运行一个Vertex AI AutoSxS Pipeline，通过生成的判断生成，评估两个候选模型。
- 打印判断和评估指标。
- 清理在此笔记本中创建的资源。

### 数据集

本教程使用的数据集是[极端摘要（XSum）](https://huggingface.co/datasets/EdinburghNLP/xsum?row=2#dataset-card-for-xsum)。该数据集包含BBC文章和配套的单句摘要，通常由文章的作者专业撰写。该数据集共有226,711篇文章，分为训练集（90%，204,045篇）、验证集（5%，11,332篇）和测试集（5%，11,334篇）。

#### 引文

In [ ]:
# @InProceedings{xsum-emnlp,
#   author =      "Shashi Narayan and Shay B. Cohen and Mirella Lapata",
#   title =       "Don't Give Me the Details, Just the Summary! {T}opic-Aware Convolutional Neural Networks for Extreme Summarization",
#   booktitle =   "Proceedings of the 2018 Conference on Empirical Methods in Natural Language Processing ",
#   year =        "2018",
#   address =     "Brussels, Belgium",
# }

成本

本教程使用了谷歌云的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)，
和[Cloud Storage价格](https://cloud.google.com/storage/pricing)，
并使用[定价计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

开始吧

### 为 Python 安装 Vertex AI SDK 和其他所需的软件包

In [ ]:
! pip3 install --upgrade --quiet \
    google-cloud-aiplatform \
    google-cloud-pipeline-components \
    gcsfs \
    datasets \
    fsspec==2023.9.2

重新启动运行环境（仅限Colab）

为了使用新安装的软件包，您必须在Google Colab上重新启动运行环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning"> <b>⚠️内核将重新启动。在继续下一步之前，请等待完成。⚠️</b> </div>

### 在Colab上验证您的笔记本环境

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置 Google Cloud 项目信息并初始化 Python 的 Vertex AI SDK

要开始使用 Vertex AI，您必须拥有一个现有的 Google Cloud 项目并[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

UUID

定义一个UUID生成函数，以避免笔记本中创建的资源名称冲突。

In [ ]:
import random
import string


def generate_uuid(length: int = 8) -> str:
    """Generate a uuid of a specified length (default=8)."""
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 创建一个云存储桶

创建一个存储桶，用于存储 AutoSxS 管道的中间产物。

In [ ]:
BUCKET_URI = "gs://[your-bucket-name-unique]"  # @param {type:"string"}

如果云存储桶尚不存在，请创建它。

In [ ]:
if (
    BUCKET_URI == ""
    or BUCKET_URI is None
    or BUCKET_URI == "gs://[your-bucket-name-unique]"
):
    BUCKET_URI = "gs://" + PROJECT_ID + "-aip-" + UUID

! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

### 导入库

导入Vertex AI Python SDK和其他必需的Python库。

In [ ]:
import os

import pandas as pd
from google.cloud import aiplatform
from google_cloud_pipeline_components.v1 import model_evaluation
from kfp import compiler

### 初始化 Python 的 Vertex AI SDK

为您的项目和对应的存储桶初始化 Python 的 Vertex SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### 教程

### 为AutoSxS生成评估数据集

在下面，您可以创建您的数据集，指定要评估的提示集。

在这个笔记本中，我们：
- 从公共资源下载Extreme Summarization（XSum）数据集。
- 使用原始数据集中的10个示例来为AutoSxS创建评估数据集。
  - `document`列中的数据将被视为模型提示。
  - `summary`列中的数据将被视为模型B的响应，因为在这个笔记本中，模型B是第三方模型。
- 将其存储为JSON文件在云存储中。

#### **注意：为了获得最佳结果，我们建议至少使用100个示例。超过400个示例后，效果会递减。**

In [ ]:
import datasets

# Download the dataset.
raw_datasets = datasets.load_dataset("xsum", split="train")

# Fetch 10 examples from the original dataset.
datasets_10 = raw_datasets.select(range(40, 50))
print("dataset structure: \n", datasets_10)

# Create the evaluation dataset with 10 examples.
prompts = datasets_10["document"]
summaries = datasets_10["summary"]
examples = pd.DataFrame({"content": prompts, "summary": summaries})

examples.head()

#### [可选] 从云存储加载您的JSONL评估数据集

或者，您可以从云存储加载自己的JSONL数据集。

In [ ]:
# # Uncomment to read from Cloud Storage.
# GCS_PATH = 'gs://your-own-evaluation-dataset.jsonl'
# examples = pd.read_json(GCS_PATH, lines=True)

#### 将您的数据集上传到云存储

最后，我们将我们的评估数据集上传到云存储中，以供AutoSxS使用。

In [ ]:
examples.to_json("evaluation_dataset.json", orient="records", lines=True)
! gsutil cp evaluation_dataset.json $BUCKET_URI/input/evaluation_dataset.json
DATASET = f"{BUCKET_URI}/input/evaluation_dataset.json"

### 创建并运行 AutoSxS 作业

为了运行 AutoSxS，我们需要定义一个带有以下参数的 `autosxs_pipeline` 作业。有关 AutoSxS 管道配置的更多详细信息可以在[这里](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.9.0/api/preview/model_evaluation.html#preview.model_evaluation.autosxs_pipeline)找到。

**必需参数:**
  - **evaluation_dataset:** 包含评估示例的 JSONL 数据集的云存储路径列表。
  - **task:** 以 {task}@{version} 的形式的评估任务。任务可以是"summarization"或者"question_answering"之一。版本是一个带有3位数字或"latest"的整数。例如: summarization@001 或 question_answering@latest。
  - **id_columns:** 区分唯一评估示例的列。
  - **autorater_prompt_parameters:** 将 autorater 提示参数映射到列或模板。预期的参数有:
      - inference_instruction - 执行任务的详细信息。
      - inference_context - 执行任务时参考的内容。

此外，我们需要指定候选模型（模型 A 和模型 B）预测结果的来源。AutoSxS 可以通过运行 Vertex Batch Prediction 来获取预测结果，或者在评估数据集中提供预定义的预测列。

**如果使用 Batch Prediction（假设使用模型 A）的模型参数:**
  - **model_a:** 完全合格的模型资源名称。如果指定了模型 A 的响应，则此参数是可选的。
  - **model_a_prompt_parameters:** 将模型 A 提示模板参数映射到列或模板。在 [text-bison](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/text#request_body) 中，唯一需要的参数是 `prompt`。
  - **model_a_parameters:** 控制从模型 A 获取预测结果的参数，例如模型温度。

**如果使用自己的预测结果（假设使用模型 A）的模型参数:**
  - **response_column_a:** 包含模型 A 响应的列。如果为模型 A 提供任何响应表，则此参数是必需的。

最后，还有一些参数用于配置额外功能，例如导出判断结果或将判断结果与人类偏好数据集进行比较，以检查 AutoRater 与人类评分者之间的一致性。
  - **judgments_format:** 写入判断结果的格式。可以是 'json' 或 'bigquery'。
  - **bigquery_destination_prefix:** 如果指定的格式是 'bigquery'，则要将判断结果写入的 BigQuery 表。
  - **human_preference_column:** 包含真实标签的列。仅在用户想要检查 autorater 的一致性与人类偏好时才需要。

在这个笔记本中，我们将使用内置的摘要指令，评估第三方模型的预测（位于`DATASET`的`summary`列中）与`text-bison@001`的输出之间的差异。正在执行的任务是摘要。

首先，在本地编译AutoSxS管道。

In [ ]:
template_uri = "pipeline.yaml"
compiler.Compiler().compile(
    pipeline_func=model_evaluation.autosxs_pipeline,
    package_path=template_uri,
)

以下代码启动一个顶点管道作业，可以在顶点用户界面中查看。该管道作业将耗时约15分钟。

这里的日志将包含当前管道的URL，因此您可以跟踪管道的进展并访问/查看管道的输出。

In [ ]:
display_name = f"autosxs-summarization-{generate_uuid()}"
prompt_column = "content"
response_column_b = "summary"
DATASET = f"{BUCKET_URI}/input/evaluation_dataset.json"
parameters = {
    "evaluation_dataset": DATASET,
    "id_columns": [prompt_column],
    "autorater_prompt_parameters": {
        "inference_context": {"column": prompt_column},
        "inference_instruction": {"template": "{{ default_instruction }}"},
    },
    "task": "summarization",
    "model_a": "publishers/google/models/text-bison@001",
    "model_a_prompt_parameters": {
        "prompt": {
            "template": "{{ default_instruction }}: {{" + prompt_column + "}}.",
            # 'template': 'Summarize the following: {{' + prompt_column + "}}.",  - This is also okay.
        },
    },
    "response_column_b": response_column_b,
}

job = aiplatform.PipelineJob(
    job_id=display_name,
    display_name=display_name,
    pipeline_root=os.path.join(BUCKET_URI, display_name),
    template_path=template_uri,
    parameter_values=parameters,
    enable_caching=False,
)
job.run()

### 获取判断和AutoSxS胜率指标
接下来，我们可以从已完成的AutoSxS任务中加载判断。

结果将被写入您在AutoSxS任务请求中指定的云存储输出桶中。

In [ ]:
# To use an existing pipeline, override job using the line below.
# job = aiplatform.PipelineJob.get('projects/[PROJECT_NUMBER]/locations/[LOCATION]/pipelineJobs/[PIPELINE_RUN_NAME]')

for details in job.task_details:
    if details.task_name == "online-evaluation-pairwise":
        break

# Judgments
judgments_uri = details.outputs["judgments"].artifacts[0].uri
judgments_df = pd.read_json(judgments_uri, lines=True)
judgments_df.head()

如果任何示例在AutoSxS中未能获得结果，它们的错误消息将被存储在错误表中。如果错误表为空，说明在评估过程中没有失败的示例。

In [ ]:
for details in job.task_details:
    if details.task_name == "online-evaluation-pairwise":
        break

# Error table
error_messages_uri = details.outputs["error_messages"].artifacts[0].uri
errors_df = pd.read_json(error_messages_uri, lines=True)
errors_df.head()

我们还可以查看从判断中计算出的衡量标准。AutoSxS输出胜率来显示一个模型胜过另一个模型的频率。

In [ ]:
# Metrics
for details in job.task_details:
    if details.task_name == "model-evaluation-text-generation-pairwise":
        break
pd.DataFrame([details.outputs["autosxs_metrics"].artifacts[0].metadata])

清理

要清理此项目中使用的所有谷歌云资源，您可以删除用于教程的[谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

将`delete_bucket`设置为**True**以删除云存储存储桶。

In [ ]:
import os

job.delete()

# Delete Cloud Storage objects that were created
delete_bucket = False
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI